In [94]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
from scipy.stats.mstats import winsorize
from sqlalchemy import create_engine
import warnings
warnings.filterwarnings('ignore')
sns.set(style="whitegrid")

In [95]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'useducation'

In [96]:
engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(postgres_user,
        postgres_pw, postgres_host, postgres_port, postgres_db))

In [97]:
df = pd.read_sql_query('select * from useducation',con=engine)
engine.dispose()
df.info()
df.groupby('YEAR').sum().mean()[12]-df.groupby('YEAR').sum().mean()[13]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1492 entries, 0 to 1491
Data columns (total 25 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   PRIMARY_KEY                   1492 non-null   object 
 1   STATE                         1492 non-null   object 
 2   YEAR                          1492 non-null   int64  
 3   ENROLL                        1229 non-null   float64
 4   TOTAL_REVENUE                 1280 non-null   float64
 5   FEDERAL_REVENUE               1280 non-null   float64
 6   STATE_REVENUE                 1280 non-null   float64
 7   LOCAL_REVENUE                 1280 non-null   float64
 8   TOTAL_EXPENDITURE             1280 non-null   float64
 9   INSTRUCTION_EXPENDITURE       1280 non-null   float64
 10  SUPPORT_SERVICES_EXPENDITURE  1280 non-null   float64
 11  OTHER_EXPENDITURE             1229 non-null   float64
 12  CAPITAL_OUTLAY_EXPENDITURE    1280 non-null   float64
 13  GRA

14021.961538461503

In [98]:
df.groupby('YEAR').sum().mean()[12]-df.groupby('YEAR').sum().mean()[13]

14021.961538461503

In [99]:
fillcols = []
fulllist = df.columns

#Generate a list of the column names with nulls in a list called fillcol
for i in range(df.shape[1]):
    if df[fulllist[i]].isnull().sum() > 0:
            fillcols.append(fulllist[i])
print(fillcols)
#Fill null values with the mean from each respective column
states = df["STATE"].unique()

for state in states:
   df.loc[df["STATE"] == state, fillcols] = df.loc[df["STATE"] == state, fillcols].interpolate()


#Drop na for avg test score columns since they over 60% empty
df.dropna(inplace=True)


['ENROLL', 'TOTAL_REVENUE', 'FEDERAL_REVENUE', 'STATE_REVENUE', 'LOCAL_REVENUE', 'TOTAL_EXPENDITURE', 'INSTRUCTION_EXPENDITURE', 'SUPPORT_SERVICES_EXPENDITURE', 'OTHER_EXPENDITURE', 'CAPITAL_OUTLAY_EXPENDITURE', 'GRADES_PK_G', 'GRADES_KG_G', 'GRADES_4_G', 'GRADES_8_G', 'GRADES_12_G', 'GRADES_1_8_G', 'GRADES_9_12_G', 'GRADES_ALL_G', 'AVG_MATH_4_SCORE', 'AVG_MATH_8_SCORE', 'AVG_READING_4_SCORE', 'AVG_READING_8_SCORE']


1. Compute the average score for each row for all students weighted properly by grade. Notice that the number of students in the 4th grade isn't the same as the number of students in the 8th grade. So, you should appropriately weigh the scores.

In [100]:
df.head()

PRIMARY_KEY        STATE  YEAR    ENROLL  TOTAL_REVENUE  \
54      1993_ARIZONA      ARIZONA  1993  676297.0      3427976.0   
55     1993_ARKANSAS     ARKANSAS  1993  311432.0      1346909.0   
57     1993_COLORADO     COLORADO  1993  539538.0      3058326.0   
58  1993_CONNECTICUT  CONNECTICUT  1993  471918.0      4064158.0   
59     1993_DELAWARE     DELAWARE  1993  104355.0       683954.0   

    FEDERAL_REVENUE  STATE_REVENUE  LOCAL_REVENUE  TOTAL_EXPENDITURE  \
54         318465.0      1415407.0      1694104.0          3623946.0   
55         128196.0       771079.0       447634.0          1376067.0   
57         147793.0      1242337.0      1668196.0          3028305.0   
58         149204.0      1433539.0      2481415.0          4079943.0   
59          44497.0       443250.0       196207.0           694534.0   

    INSTRUCTION_EXPENDITURE  ...  GRADES_4_G  GRADES_8_G  GRADES_12_G  \
54                1578889.0  ...     57701.0     53500.0      37614.0   
55                 782791.0  ...     34255.0     36471.0      27169.0   
57                1537714.0  ...     50807.0     47665.0      34770.0   
58                2302852.0  ...     39093.0     34662.0      27979.0   
59                 394680.0  ...      8285.0      8265.0       5882.0   

    GRADES_1_8_G  GRADES_9_12_G  GRADES_ALL_G  AVG_MATH_4_SCORE  \
54      461398.0       182737.0      647299.0        215.834434   
55      280280.0       125801.0      407329.0        211.616130   
57      403871.0       164260.0      575380.0        222.219036   
58      310022.0       127655.0      443893.0        228.105503   
59       67899.0        28930.0       97394.0        217.181255   

    AVG_MATH_8_SCORE  AVG_READING_4_SCORE  AVG_READING_8_SCORE  
54        265.993417           206.291984           262.448330  
55        257.647254           208.685273           266.423396  
57        273.200878           215.523979           266.856804  
58        275.202320           224.712775           264.457874  
59        263.834857           207.792430           262.255809  

[5 rows x 25 columns]

In [101]:
G4frac = df['GRADES_4_G']/(df['GRADES_4_G']+ df['GRADES_8_G'])
G8frac = df['GRADES_8_G']/(df['GRADES_4_G']+ df['GRADES_8_G'])
#Avg math
weightedmath= (df['AVG_MATH_4_SCORE']*G4frac)+(df['AVG_MATH_8_SCORE']*G8frac)


#Avg reading
weightedread= (df['AVG_READING_4_SCORE']*G4frac)+(df['AVG_READING_8_SCORE']*G8frac)
df['weighted combined']= (weightedmath+weightedread)/2
#df.drop(['weighted mean'], axis = 1, inplace=True)
df.groupby('YEAR').sum().mean()[12]-df.groupby('YEAR').sum().mean()[13]

-5989.339999999851

2. What are the correlations between this newly created score variable and the expenditure types? Which 1 of the expenditure types is more correlated than the others?

In [102]:
dfexpscore = df.iloc[0:, [8,9,10,11,12,25]]
dfexpscore.corr()

TOTAL_EXPENDITURE  INSTRUCTION_EXPENDITURE  \
TOTAL_EXPENDITURE                      1.000000                 0.990829   
INSTRUCTION_EXPENDITURE                0.990829                 1.000000   
SUPPORT_SERVICES_EXPENDITURE           0.993175                 0.975867   
OTHER_EXPENDITURE                      0.946483                 0.907235   
CAPITAL_OUTLAY_EXPENDITURE             0.928576                 0.889763   
weighted combined                      0.093020                 0.105787   

                              SUPPORT_SERVICES_EXPENDITURE  OTHER_EXPENDITURE  \
TOTAL_EXPENDITURE                                 0.993175           0.946483   
INSTRUCTION_EXPENDITURE                           0.975867           0.907235   
SUPPORT_SERVICES_EXPENDITURE                      1.000000           0.953789   
OTHER_EXPENDITURE                                 0.953789           1.000000   
CAPITAL_OUTLAY_EXPENDITURE                        0.913680           0.924197   
weighted combined                                 0.098643           0.027635   

                              CAPITAL_OUTLAY_EXPENDITURE  weighted combined  
TOTAL_EXPENDITURE                               0.928576           0.093020  
INSTRUCTION_EXPENDITURE                         0.889763           0.105787  
SUPPORT_SERVICES_EXPENDITURE                    0.913680           0.098643  
OTHER_EXPENDITURE                               0.924197           0.027635  
CAPITAL_OUTLAY_EXPENDITURE                      1.000000           0.010670  
weighted combined                               0.010670           1.000000

The Instruction expenditure has low but the most correlation.

3. Now, apply PCA to the 4 expenditure types. How much of the total variance is explained by the 1st component?

In [103]:

from sklearn.decomposition import PCA 
from sklearn.preprocessing import StandardScaler

dfexp = df.iloc[0:, [8,9,10,11,12]]
type(dfexp)
dfexpstd = StandardScaler().fit_transform(dfexp)

pca = PCA(n_components=1)
dfexpscore['pca'] = pca.fit_transform(dfexpstd)

print(pca.explained_variance_ratio_)

[0.9541168]


4. What is the correlation between the overall score variable and the 1st principal component?

In [104]:
dfexpscore.corr()

TOTAL_EXPENDITURE  INSTRUCTION_EXPENDITURE  \
TOTAL_EXPENDITURE                      1.000000                 0.990829   
INSTRUCTION_EXPENDITURE                0.990829                 1.000000   
SUPPORT_SERVICES_EXPENDITURE           0.993175                 0.975867   
OTHER_EXPENDITURE                      0.946483                 0.907235   
CAPITAL_OUTLAY_EXPENDITURE             0.928576                 0.889763   
weighted combined                      0.093020                 0.105787   
pca                                    0.995210                 0.975856   

                              SUPPORT_SERVICES_EXPENDITURE  OTHER_EXPENDITURE  \
TOTAL_EXPENDITURE                                 0.993175           0.946483   
INSTRUCTION_EXPENDITURE                           0.975867           0.907235   
SUPPORT_SERVICES_EXPENDITURE                      1.000000           0.953789   
OTHER_EXPENDITURE                                 0.953789           1.000000   
CAPITAL_OUTLAY_EXPENDITURE                        0.913680           0.924197   
weighted combined                                 0.098643           0.027635   
pca                                               0.990651           0.968760   

                              CAPITAL_OUTLAY_EXPENDITURE  weighted combined  \
TOTAL_EXPENDITURE                               0.928576           0.093020   
INSTRUCTION_EXPENDITURE                         0.889763           0.105787   
SUPPORT_SERVICES_EXPENDITURE                    0.913680           0.098643   
OTHER_EXPENDITURE                               0.924197           0.027635   
CAPITAL_OUTLAY_EXPENDITURE                      1.000000           0.010670   
weighted combined                               0.010670           1.000000   
pca                                             0.952869           0.069272   

                                   pca  
TOTAL_EXPENDITURE             0.995210  
INSTRUCTION_EXPENDITURE       0.975856  
SUPPORT_SERVICES_EXPENDITURE  0.990651  
OTHER_EXPENDITURE             0.968760  
CAPITAL_OUTLAY_EXPENDITURE    0.952869  
weighted combined             0.069272  
pca                           1.000000

Correlation is pretty low at ~ 0.07

5. If you were to choose the best variables for your model, would you prefer using the 1st principal component instead of the expenditure variables? Why?

Instruction expenditure is higher than PCA but not by much. Neither work that great